# Imports & Mounts

In [ ]:
from google.colab import drive
drive.mount('/gdrive')


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%%bash
cat > requirements.txt << EOF
# Standardimports
numpy
pandas
xlrd
scipy
matplotlib
seaborn
datetime
IPython
sklearn

# NLP mit Spacy und deutschem Language Model
spacy>=2.2.0,<3.0.0
https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-2.2.5/de_core_news_sm-2.2.5.tar.gz#egg=de_core_news_sm

EOF

pip install -r requirements.txt

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle

import spacy
from spacy.lang.de import German
from spacy import displacy
from spacy.lang.de.stop_words import STOP_WORDS
# nlp = spacy.load('de_core_news_md')
nlp = spacy.load('de_core_news_sm')
# nlp = German()

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.cluster import KMeans

In [ ]:
path = '/gdrive/My Drive/data/'
le = LabelEncoder()

gnd = pd.read_csv(path+'10kGNAD_train.csv', header=None, sep=';', quotechar="'", names=['labelEnc', 'text'])
pd.set_option('display.max_colwidth', None)
gnd['label'] = le.fit_transform(gnd.labelEnc)
gnd = gnd.iloc[:,[2,0,1]]
gnd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9245 entries, 0 to 9244
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   label     9245 non-null   int64 
 1   labelEnc  9245 non-null   object
 2   text      9245 non-null   object
dtypes: int64(1), object(2)
memory usage: 216.8+ KB


# Preprocessing Dataset

In [ ]:
def textcleaner(string):
    doc = nlp(string)
    string_clean = [w.lemma_ for w in doc if (not w.is_stop and not w.is_punct and not w.like_num)]
    return string_clean

In [ ]:
gnd["text120"] = (gnd.text.str.split().str[:120]).apply(", ".join)
gnd["prepro"] = gnd.text.apply(textcleaner).apply(", ".join)
gnd.head()

label  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  prepro
0      5  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
def SavePickle(name, object):
    save_df = open(str(name)+'.pickle','wb')
    pickle.dump(object,save_df)
    save_df.close()

path = '/gdrive/My Drive/data/'
SavePickle(path+'10kGNAD_preprocessed', gnd)

# Preprocessed 10kGND einlesen (pickle-Objekt)

In [ ]:
path = '/gdrive/My Drive/data/'
os.listdir(path)

['TIMEGenderData.csv',
 'times_magazin.csv',
 'noten.csv',
 '10kGNAD_train.csv',
 '10kGNAD_preprocessed.pickle']

In [3]:
FILE = '10kGNAD_preprocessed'
# open_df = open(path+FILE+'.pickle','rb')
open_df = open(FILE+'.pickle','rb')
gnd = pickle.load(open_df)
open_df.close()


le = LabelEncoder()
gnd.label = le.fit_transform(gnd.labelEnc)
gnd.info()
print("Preprocessed Datenset 10k German News in Dataframe gnd geladen")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9245 entries, 0 to 9244
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   label     9245 non-null   int32 
 1   labelEnc  9245 non-null   object
 2   text      9245 non-null   object
 3   text120   9245 non-null   object
 4   prepro    9245 non-null   object
dtypes: int32(1), object(4)
memory usage: 325.1+ KB
Preprocessed Datenset 10k German News in Dataframe gnd geladen


# SVC Classification auf voll gelabeltem Datenset

In [ ]:
txt = "text"  # Alternative Methodik mit "prepro" := legt fest, ob auf Rawtext (text) oder Prepro-Text (prepro) trainiert wird

X_train, X_test, y_train, y_test = train_test_split(gnd[txt], gnd["label"], test_size=.3, stratify=gnd.label)
print(X_train.shape, X_test.shape)


(6471,) (2774,)


In [ ]:
# classifier pipeline mit Support Vektor Machine als Classifier definieren
lsvc_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('clf', LinearSVC(dual=False, C=1.6, class_weight="balanced")),
])

# classifier mit vorgeschalteter Pipeline trainieren:
lsvc_classifier.fit(X_train, y_train)  # train the classifier

# trainierten classifier auf die Testdaten anwenden, um den Accuracy Score zu ermitteln:
predicted = lsvc_classifier.predict(X_test)  # predict the test set 

score_report = classification_report(predicted, y_test, target_names=le.classes_.tolist(), output_dict=True)
scores_SVC_FullData = pd.DataFrame(score_report).transpose()
scores_SVC_FullData

,precision,recall,f1-score,support
Etat,0.772222,0.902597,0.832335,154.000000
Inland,0.755474,0.784091,0.769517,264.000000
International,0.872549,0.896725,0.884472,397.000000
Kultur,0.917808,0.837500,0.875817,160.000000
Panorama,0.845475,0.804622,0.824543,476.000000
Sport,0.959877,0.984177,0.971875,316.000000
Web,0.887417,0.885463,0.886439,454.000000
Wirtschaft,0.850394,0.778846,0.813049,416.000000
Wissenschaft,0.812903,0.919708,0.863014,137.000000
accuracy,0.858688,0.858688,0.858688,0.858688


# Active Labeling mit 50 Trainingsdaten auf Basis k-Means (k=50)

In [ ]:
k = 100

km = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('kmeans', KMeans(n_clusters = k, n_jobs=-1,  )),
])


X_digits_dist = km.fit_transform(X_train) # X_digits_dist: Distanz jedes Datensatzes zu den 50 Centroids: Shape: n / k

# Repräsentative Datenpunkte (sind am nähesten zum Centroid des jeweiligen k-Clusters)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = X_train.iloc[representative_digit_idx]
y_representative_digits = y_train.iloc[representative_digit_idx]

print(X_representative_digits.shape, y_representative_digits.shape)


(100,) (100,)


In [ ]:
# classifier pipeline mit Support Vektor Machine als Classifier definieren
lsvc_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('clf', LinearSVC(dual=False, C=1.6, class_weight="balanced")),
])

# classifier mit vorgeschalteter Pipeline trainieren:
lsvc_classifier.fit(X_representative_digits, y_representative_digits)  

# trainierten classifier auf die Testdaten anwenden, um den Accuracy Score zu ermitteln:
predicted = lsvc_classifier.predict(X_test)  

score_report = classification_report(predicted, y_test, target_names=le.classes_.tolist(), output_dict=True)
scores_SVC_50Labels = pd.DataFrame(score_report).transpose()
scores_SVC_50Labels


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
Etat,0.133333,1.000000,0.235294,24.000000
Inland,0.697080,0.434091,0.535014,440.000000
International,0.779412,0.637275,0.701213,499.000000
Kultur,0.000000,0.000000,0.000000,0.000000
Panorama,0.785872,0.489011,0.602879,728.000000
Sport,0.904321,0.794038,0.845599,369.000000
Web,0.766004,0.755991,0.760965,459.000000
Wirtschaft,0.433071,0.647059,0.518868,255.000000
Wissenschaft,0.000000,0.000000,0.000000,0.000000
accuracy,0.610671,0.610671,0.610671,0.610671


# Active Labeling mit propagierten Labels
Basis: 50 gelabelte Ausgangsdatenpunkte und k-Means mit k = 50

Labels propagieren:

In [ ]:
y_train_propagated = np.empty(len(X_train), dtype=np.int32)  # arbitrary filler data

for idx, lbl in enumerate(y_representative_digits):          # ruft für jeden der 50 Cluster das entsprechende repräsentative Label auf
    y_train_propagated[km["kmeans"].labels_==idx] = lbl      # Leerer Vektor wird mit dem jeweils repräsentativen Labelwert befüllt (lbl)
    # print(y_train_propagated[km["kmeans"].labels_==idx])

print("Umfang propagiertes Labelset: ", y_train_propagated.shape, "\n")
pd.Series(y_train_propagated).value_counts(normalize=True)  # 

Umfang propagiertes Labelset:  (6471,) 



4    0.193788
0    0.152218
1    0.146809
6    0.145263
2    0.132437
7    0.128728
5    0.097976
8    0.002782
dtype: float64

Gesamtes Datenset X_train auf die propagierten Labels fitten (y_train_propagated).
Dann auf Testdatenset anwenden

In [ ]:
# classifier pipeline mit Support Vektor Machine als Classifier definieren
lsvc_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('clf', LinearSVC(dual=False, C=1.6, class_weight="balanced")),
])

# classifier mit vorgeschalteter Pipeline trainieren:
lsvc_classifier.fit(X_train, y_train_propagated)  

# trainierten classifier auf die Testdaten anwenden, um den Accuracy Score zu ermitteln:
predicted = lsvc_classifier.predict(X_test)  

score_report = classification_report(predicted, y_test, target_names=le.classes_.tolist(), output_dict=True)
scores_SVC_AllProp50Labels = pd.DataFrame(score_report).transpose()
scores_SVC_AllProp50Labels


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
Etat,0.666667,0.279070,0.393443,430.000000
Inland,0.671533,0.438095,0.530259,420.000000
International,0.708333,0.809524,0.755556,357.000000
Kultur,0.000000,0.000000,0.000000,0.000000
Panorama,0.724062,0.631985,0.674897,519.000000
Sport,0.882716,0.993056,0.934641,288.000000
Web,0.810155,0.851508,0.830317,431.000000
Wirtschaft,0.509186,0.593272,0.548023,327.000000
Wissenschaft,0.012903,1.000000,0.025478,2.000000
accuracy,0.638068,0.638068,0.638068,0.638068


# Active Labeling  mit 20% der zentralsten Datenpunkte
Basis: Nur die 20% nähestgelegenen Datenpunkte zum Centroid werden mit dem Label des nähesgelegenen Datenpunktes gelabelt

In [ ]:
percentile_closest = 20

X_cluster_dist = X_digits_dist[np.arange(len(X_train)), km["kmeans"].labels_]
for i in range(k):
    in_cluster = (km["kmeans"].labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1

partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = X_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]
X_train_partially_propagated.shape, y_train_partially_propagated.shape

((1334,), (1334,))

In [ ]:
# classifier pipeline mit Support Vektor Machine als Classifier definieren
lsvc_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('clf', LinearSVC(dual=False, C=1.6, class_weight="balanced")),
])

# classifier mit vorgeschalteter Pipeline trainieren:
lsvc_classifier.fit(X_train_partially_propagated, y_train_partially_propagated)  

# trainierten classifier auf die Testdaten anwenden, um den Accuracy Score zu ermitteln:
predicted = lsvc_classifier.predict(X_test)  

score_report = classification_report(predicted, y_test, target_names=le.classes_.tolist(), output_dict=True)
scores_SVC_AllProp50Labels = pd.DataFrame(score_report).transpose()
scores_SVC_AllProp50Labels


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision,recall,f1-score,support
Etat,0.750000,0.210280,0.328467,642.000000
Inland,0.631387,0.570957,0.599653,303.000000
International,0.664216,0.831288,0.738420,326.000000
Kultur,0.000000,0.000000,0.000000,0.000000
Panorama,0.728477,0.676230,0.701382,488.000000
Sport,0.836420,0.992674,0.907873,273.000000
Web,0.772627,0.862069,0.814901,406.000000
Wirtschaft,0.566929,0.644776,0.603352,335.000000
Wissenschaft,0.006452,1.000000,0.012821,1.000000
accuracy,0.629776,0.629776,0.629776,0.629776


# Active Label mit Voting Propagated Labels 
Basis 50 k-Means Cluster

In [ ]:
i = 0
in_cluster = (km["kmeans"].labels_ == i)
y_in_cluster = y_train.iloc[in_cluster]
# y_in_cluster
zaehler = y_in_cluster.count()
zaehlerGesamt += zaehler
modus = y_in_cluster.mode().values
anzLab = y_in_cluster.nunique()
purity = (1-(1/(zaehler/anzLab)))*100
clusterLabelTab.loc[i,:] = [i, zaehler, modus, anzLab,]
y_train_propVote[in_cluster] = modus

clusterLabelTab = clusterLabelTab.astype(int, copy=False)
clusterLabelTab.T

pd.Series(y_train_propVote).value_counts()

999    4085
6       603
7       513
4       488
5       338
1       221
2       180
0        43
dtype: int64

In [ ]:
# Majority Label Voting

y_train_propVote = np.full(  len(X_train), 999, dtype=np.int32)
zaehlerGesamt = 0
clusterLabelTab = pd.DataFrame({"cid":[], "anzahl":[], "modus":[], "anzLab":[]}) #  "purity":[]
for i in range(k):
    in_cluster = (km["kmeans"].labels_ == i)
    y_in_cluster = y_train.iloc[in_cluster]
    # y_in_cluster
    zaehler = y_in_cluster.count()
    zaehlerGesamt += zaehler
    modus = y_in_cluster.mode().values
    anzLab = y_in_cluster.nunique()
    purity = (1-(1/(zaehler/anzLab)))*100
    # clusterLabelTab.loc[i,:] = [i, zaehler, modus, anzLab,]
    y_train_propVote[in_cluster] = modus

clusterLabelTab = clusterLabelTab.astype(int, copy=False)
# clusterLabelTab.T

y_train_propVote.value_counts()

ValueError: ignored

In [ ]:
clusterLabelTab.T

In [ ]:
# classifier pipeline mit Support Vektor Machine als Classifier definieren
lsvc_classifier = Pipeline([
  ('vect', CountVectorizer()),
  ('tfidf', TfidfTransformer(sublinear_tf=True)),                   
  ('clf', LinearSVC(dual=False, C=1.6, class_weight="balanced")),
])

# classifier mit vorgeschalteter Pipeline trainieren:
lsvc_classifier.fit(X_train, y_train_propVote)  

# trainierten classifier auf die Testdaten anwenden, um den Accuracy Score zu ermitteln:
predicted = lsvc_classifier.predict(X_test)  

score_report = classification_report(predicted, y_test, target_names=le.classes_.tolist(), output_dict=True)
scores_SVC_PropVotedLabels = pd.DataFrame(score_report).transpose()
scores_SVC_PropVotedLabels
